# ASMSA: Run molecular dynamics with CV derived AAE model

**Previous steps**
- [prepare.ipynb](prepare.ipynb): Download and sanity check input files
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training|
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule



In [ ]:
# %cd villin

In [ ]:
!cp ../*.mdp .

In [ ]:
#avoid TF to consume GPU memory
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

In [ ]:
import asmsa
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import gromacs as gmx

In [ ]:
ompthreads=4
mpiranks=1
gpus=1

In [ ]:
# Kubernetes deployment
mdrunner=gmx.MDrunnerK8s()

def mdrun(**kwargs):
    mdrunner.run(pre={'omp':ompthreads,'mpi':mpiranks,'gpus':gpus}, mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'},ncores=mpiranks)


In [ ]:
# alternative local deployment
'''
mdrunner=gmx.run.MDrunner()

# XXX: no MPI support so far
def mdrun(**kwargs):
    mdrunner.run(mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'})
'''

In [ ]:
exec(open('inputs.py').read())

fulltop = base + '-full.top'

In [ ]:
# on fresh run only!
!cp {topol} {fulltop}

In [ ]:
# Dodecahedron box of suitable size (adjust eventually)
mdbox=2.0
gmx.editconf(f=gro,o=f"{base}-box.gro",c=True,d=str(mdbox),bt="dodecahedron")

In [ ]:
# Solvate
gmx.solvate(cp=f"{base}-box.gro",cs="spc216.gro",o=f"{base}-solv.gro",p=fulltop)

In [ ]:
# Add ions
gmx.grompp(f='md_files/ions.mdp',c=f"{base}-solv.gro",p=fulltop,o="ions.tpr")

In [ ]:
gmx.select(s="ions.tpr",on='sol.ndx',select='SOL')
gmx.genion(s="ions.tpr",n="sol.ndx",o=f"{base}-ions.gro",p=fulltop,pname="NA",nname="CL",neutral=True)

In [ ]:
# Energy minimization

#gmx.start(f"grompp -f minim-sol.mdp -c {base}-ions.gro -p {base}.top -o em.tpr",wait=True,delete=True)
gmx.grompp(f="md_files/minim-sol.mdp",c=f"{base}-ions.gro",p=fulltop,o="em.tpr")

In [ ]:
# This job may run a bit longer, start it in background, with more cores and GPU

#gmx.start(f"mdrun -v -deffnm em -pin on",cores=4,gpus=1)
mdrun(deffnm="em")

In [ ]:
# Isothermal-isochoric equilibration, following the same job pattern
gmx.grompp(f="md_files/nvt.mdp",c="em.gro",r="em.gro",p=fulltop,o="nvt.tpr")

In [ ]:
mdrun(deffnm='nvt')

In [ ]:
# And isothermal-isobaric equilibration
gmx.grompp(f='md_files/npt.mdp',c='nvt.gro',r='nvt.gro',t='nvt.cpt',p=fulltop,o='npt.tpr')

In [ ]:
mdrun(deffnm='npt')

In [ ]:
# Extract temperature, pressure and density from the trajectories and plot them
gmx.energy(f='npt.edr',o='press.xvg',input='Pressure')
gmx.energy(f='npt.edr',o='dens.xvg',input='Density')
gmx.energy(f='npt.edr',o='temp.xvg',input='Temperature')

In [ ]:
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

In [ ]:
# Now we are ready to run metadynamics with plumed.dat generated in train.ipynb
# Checkpoints are stored, hence the simulation can be restarted if it gets killed
#1 m

#mdsteps=500000000

# 500 ns
mdsteps=250000000

# 10 ns
#mdsteps=5000000

# 100 ps
#mdsteps=50000

In [ ]:
with open('md.mdp','w') as mdp:
    mdp.write(f'''
integrator              = md        ; leap-frog integrator
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 0         ; suppress bulky .trr file by specifying 
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; save energies every 10.0 ps
nstlog                  = 5000      ; update log file every 10.0 ps
nstxout-compressed      = 5000      ; save compressed coordinates every 10.0 ps
; compressed-x-grps       = System    ; save the whole system
compressed-x-grps = Protein
; Bond parameters
continuation            = yes       ; Restarting after NPT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Neighborsearching
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 

nsteps = {mdsteps}
''')

In [ ]:
gmx.grompp(f='md.mdp',c='../Thermal-unoflding/trpcage_md_data/npt.gro',t='../Thermal-unoflding/trpcage_md_data/npt.cpt',p=fulltop,o='md.tpr')

In [ ]:
!touch HILLS
#mdrun(deffnm='md',plumed='plumed.dat',cpi='md.cpt')
#mdrun(deffnm='md')

In [ ]:
mdrunner.prehook(mpi=4,omp=4,gpus=1)

In [ ]:
gmx.trjconv(f='md.xtc',s=base+'-box.gro',pbc='nojump',input='Protein Protein'.split(),o='pbc.xtc')

In [ ]:
gmx.trjconv(f='pbc.xtc',s=base+'-box.gro',fit='rot+trans',input='Protein Protein'.split(),o='fit.xtc')

In [ ]:
import mdtraj as md
tr = md.load('fit.xtc',top=base+'-box.gro')

In [ ]:
import nglview as nv
nv.show_mdtraj(tr)

In [ ]:
mdrunner.prehook(mpi=2,omp=4,gpus=1)

In [ ]:
gmx.convert_tpr(s='md.tpr',o='md-new.tpr',extend=100000)
!mv md-new.tpr md.tpr

In [ ]:
mdrun(deffnm='md',plumed='plumed.dat',cpi='md.cpt')